In [1]:
import sys
sys.path.insert(0, "/home/uih/JYL/Programs/YOLO/")
import torch
from models import Yolov5SmallWithPlainBscp

In [2]:
class_num = 20

In [30]:
yolo = Yolov5SmallWithPlainBscp(3, class_num)
my_state_dict = yolo.state_dict()

In [31]:
for m in yolo.modules():
    if isinstance(m, torch.nn.Conv2d):
        print(m.weight.data.shape)

torch.Size([32, 12, 3, 3])
torch.Size([64, 32, 3, 3])
torch.Size([32, 64, 1, 1])
torch.Size([32, 64, 1, 1])
torch.Size([32, 32, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([32, 32, 1, 1])
torch.Size([32, 32, 3, 3])
torch.Size([128, 64, 3, 3])
torch.Size([64, 128, 1, 1])
torch.Size([64, 128, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([128, 128, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3])
torch.Size([256, 128, 3, 3])
torch.Size([128, 256, 1, 1])
torch.Size([128, 256, 1, 1])
torch.Size([128, 128, 1, 1])
torch.Size([256, 256, 1, 1])
torch.Size([128, 128, 1, 1])
torch.Size([128, 128, 3, 3])
torch.Size([128, 128, 1, 1])
torch.Size([128, 128, 3, 3])
torch.Size([128, 128, 1, 1])
torch.Size([128, 128, 3, 3])
torch.Size([512, 256, 3, 3])
torch.Size([256, 512, 1, 1])
torch.Size([512, 1024, 1, 1])
torch.Size([256, 512, 1, 1])
torch.Size([256, 512, 1, 1])
torch.Size([256, 

In [19]:
my_total_k = []
my_total_v = []
my_focus_layer = []
my_stage_1_conv = []
for i, (k, v) in enumerate(my_state_dict.items()):
    my_total_k.append(k)
    my_total_v.append(v)
    if "focus" in k:
        my_focus_layer.append(v)
        continue
    if "stage1_conv" in k:
        my_stage_1_conv.append(v)
        
    print(f'{i}: {k}\t\t{v.shape}')

6: backbone_stage1_conv.conv.weight		torch.Size([64, 32, 3, 3])
7: backbone_stage1_conv.bn.weight		torch.Size([64])
8: backbone_stage1_conv.bn.bias		torch.Size([64])
9: backbone_stage1_conv.bn.running_mean		torch.Size([64])
10: backbone_stage1_conv.bn.running_var		torch.Size([64])
11: backbone_stage1_conv.bn.num_batches_tracked		torch.Size([])
12: backbone_stage1_bscp.conv_bn_act_1.conv.weight		torch.Size([32, 64, 1, 1])
13: backbone_stage1_bscp.conv_bn_act_1.bn.weight		torch.Size([32])
14: backbone_stage1_bscp.conv_bn_act_1.bn.bias		torch.Size([32])
15: backbone_stage1_bscp.conv_bn_act_1.bn.running_mean		torch.Size([32])
16: backbone_stage1_bscp.conv_bn_act_1.bn.running_var		torch.Size([32])
17: backbone_stage1_bscp.conv_bn_act_1.bn.num_batches_tracked		torch.Size([])
18: backbone_stage1_bscp.conv_2.weight		torch.Size([32, 64, 1, 1])
19: backbone_stage1_bscp.conv_1.weight		torch.Size([32, 32, 1, 1])
20: backbone_stage1_bscp.conv_bn_act_2.conv.weight		torch.Size([64, 64, 1, 1])
21: bac

In [20]:
v5_state_dict = torch.load('/home/uih/JYL/Programs/Yolov5Base/checkpoints/every_for_coco_small_with_plain_bscp.pth', map_location='cpu')

In [21]:
v5_state_dict = v5_state_dict['model']

In [22]:
v5_total_k, v5_total_v = [], []
v5_focus_layer = []
v5_stage_1_conv = []
for i, (k, v) in enumerate(v5_state_dict.items()):
    if 'anchor' not in k:
        v5_total_k.append(k)
        v5_total_v.append(v)
        print(f'{i}: {k}\t\t{v.shape}')
    if "model.0" in k:
        v5_focus_layer.append(v)
    if "model.1" in k:
        v5_stage_1_conv.append(v)

0: focus.conv_bn_act.conv.weight		torch.Size([32, 12, 3, 3])
1: focus.conv_bn_act.bn.weight		torch.Size([32])
2: focus.conv_bn_act.bn.bias		torch.Size([32])
3: focus.conv_bn_act.bn.running_mean		torch.Size([32])
4: focus.conv_bn_act.bn.running_var		torch.Size([32])
5: focus.conv_bn_act.bn.num_batches_tracked		torch.Size([])
6: backbone_stage1_conv.conv.weight		torch.Size([64, 32, 3, 3])
7: backbone_stage1_conv.bn.weight		torch.Size([64])
8: backbone_stage1_conv.bn.bias		torch.Size([64])
9: backbone_stage1_conv.bn.running_mean		torch.Size([64])
10: backbone_stage1_conv.bn.running_var		torch.Size([64])
11: backbone_stage1_conv.bn.num_batches_tracked		torch.Size([])
12: backbone_stage1_bscp.conv_bn_act_1.conv.weight		torch.Size([32, 64, 1, 1])
13: backbone_stage1_bscp.conv_bn_act_1.bn.weight		torch.Size([32])
14: backbone_stage1_bscp.conv_bn_act_1.bn.bias		torch.Size([32])
15: backbone_stage1_bscp.conv_bn_act_1.bn.running_mean		torch.Size([32])
16: backbone_stage1_bscp.conv_bn_act_1.bn.ru

In [23]:
len(my_total_k), len(v5_total_k)

(368, 368)

In [24]:
for i in range(len(my_total_k)):
    if my_total_v[i].shape != v5_total_v[i].shape:
        print(i)

362
363
364
365
366
367


## load prarmeters

In [32]:
i = 0
total = v5_total_v[:]
for m in yolo.modules():
    if i >= 362:
        continue
    if isinstance(m, torch.nn.Conv2d):
        m.weight.data = total[i]
        print(m.weight.data.shape, total[i].shape)
        i += 1
        if m.bias is not None:
            m.bias.data = total[i]
            i += 1
        
    if isinstance(m, torch.nn.BatchNorm2d):
        m.weight.data = total[i]
        i += 1
        m.bias.data = total[i]
        i += 1
        m.running_mean = total[i]
        i += 1
        m.running_var = total[i]
        i += 1
        m.num_batches_tracked = total[i]
        i += 1

torch.Size([32, 12, 3, 3]) torch.Size([32, 12, 3, 3])
torch.Size([64, 32, 3, 3]) torch.Size([64, 32, 3, 3])
torch.Size([32, 64, 1, 1]) torch.Size([32, 64, 1, 1])
torch.Size([32, 64, 1, 1]) torch.Size([32, 64, 1, 1])
torch.Size([32, 32, 1, 1]) torch.Size([32, 32, 1, 1])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([32, 32, 1, 1]) torch.Size([32, 32, 1, 1])
torch.Size([32, 32, 3, 3]) torch.Size([32, 32, 3, 3])
torch.Size([128, 64, 3, 3]) torch.Size([128, 64, 3, 3])
torch.Size([64, 128, 1, 1]) torch.Size([64, 128, 1, 1])
torch.Size([64, 128, 1, 1]) torch.Size([64, 128, 1, 1])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([128, 128, 1, 1]) torch.Size([128, 128, 1, 1])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3]) torch.Size([64, 64, 3, 3])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3]) torch.Size([64, 64, 3, 3])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([64, 64

In [33]:
i

362

In [35]:
i = 0
total = v5_total_v[:]
for m in yolo.modules():
    if i >= 362:
        continue
    if isinstance(m, torch.nn.Conv2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        if m.bias is not None:
            assert (m.bias.data == total[i]).all()
            i += 1
    if isinstance(m, torch.nn.BatchNorm2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        assert (m.bias.data == total[i]).all()
        i += 1
        assert (m.running_mean == total[i]).all()
        i += 1
        assert (m.running_var == total[i]).all()
        i += 1
        assert (m.num_batches_tracked == total[i]).all()
        i += 1

In [36]:
my_output_state = yolo.state_dict()
torch.save({'model': my_output_state}, "./yolov5_plainsmall_for_voc.pth")